## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [3]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from tensorflow.python.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical

In [8]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
X = model.output
X = Dropout(0.5)(X)
output_layer = Dense(num_classes, activation='softmax', name='softmax')(X)
net_final = Model(inputs=model.input, outputs=output_layer)
net_final.summary()

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 32, 32, 16)   448         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 32, 32, 16)   64          conv2d_125[0][0]                 
__________________________________________________________________________________________________
activation_113 (Activation)     (None, 32, 32, 16)   0           batch_normalization_113[0][0]    
____________________________________

In [9]:
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [11]:
net_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = net_final.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                              steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs, verbose=1, validation_data=(x_test, y_test))
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
391/390 [==============================] - 1002s 3s/step - loss: 2.4817 - acc: 0.2104 - val_loss: 2.2654 - val_acc: 0.2947
Epoch 2/30
391/390 [==============================] - 961s 2s/step - loss: 2.1525 - acc: 0.2896 - val_loss: 2.0630 - val_acc: 0.3628
Epoch 3/30
391/390 [==============================] - 956s 2s/step - loss: 2.0179 - acc: 0.3134 - val_loss: 1.9663 - val_acc: 0.3727
Epoch 4/30
391/390 [==============================] - 956s 2s/step - loss: 1.9412 - acc: 0.3301 - val_loss: 1.7908 - val_acc: 0.4533
Epoch 5/30
391/390 [==============================] - 955s 2s/step - loss: 1.8898 - acc: 0.3404 - val_loss: 2.0996 - val_acc: 0.3490
Epoch 6/30
391/390 [==============================] - 956s 2s/step - loss: 1.8483 - acc: 0.3479 - val_loss: 1.6652 - val_acc: 0.4870
Epoch 7/30
391/390 [==============================] - 955s 2s/step - loss: 1.8152 - acc: 0.3522 - val_loss: 1.8037 - val_acc: 0.4064
Epoch 8/30
391/390 [